# 第一回レポート課題「桁落ちの例」

## 学籍番号:08B22182 氏名:平山聖輝 

（Jupyter Notebookは初めて使うので慎重にメモを残しながらレポートを進めていきます）

 講義内で紹介されていたような計算機を用いることによる「桁落ち」の例がほかにないかインターネットで調べてみた．


（参考にしたページは[こちら-うさぎ塾](https://www.momoyama-usagi.com/entry/info-calc-sys04#i-4 )）

### 二次方程式の解と桁落ち

まず，桁落ちが生じる原因が何かというと講義にもあったように桁落ちは**非常に近い値の引き算**を実行することで生じる．講義内容で触れられていた例は二次方程式の解であった．二次方程式の解は

$$   % "$"二つで囲むと数式を打てる（LATEX表記）
    ax^2 + bx+ c = 0
$$

$$     % \begin{align}...\end{align}で &= いろいろな文章様式を使えるようになるみたい
    \begin{align}
       % &= は = の位置でそろえて改行
    x_1  &= \frac{-b + \sqrt{b^2-4ac}}{2a} \\
    x_2  &= \frac{-b-\sqrt{b^2-4ac}}{2a}
       % \frac{}{} で分数，\sqrt{} でルート
    \end{align}
$$

と書くことができる．この二つ目の解 $ x_2 $ は $ b^2 >> 4ac $ のとき　$ |b| ~= \sqrt{b^2-4ac} $ となり桁落ちが生じてしまう． <br> 
この場合桁落ちしないようにするにはまず，桁落ちしていない $ x_1 $ を求めて，解と係数の関係から　$ x_2 = \frac{c}{ax_1} $ を求めればよいことも講義中に学んだ．

### 微分と桁落ち

ではほかにどんな例があるだろうか．非常に近い値の引き算を実施するほかの例を取り上げてみると**微分**があることにすぐ気づく．なぜなら　$ f(x) $ の $ x_0 $ における微分の定義は

$$
    f('x_0)= \lim_{h\to 0}  \frac{f(x_0 + h)-f(x)}{h}
$$

となり，分子が必然的に近い値の差になることが理解される．講義でも述べられていたが，数値計算する際には $ h $ を無限に小さくすることは不可能であるから　$ h=1/2, 1/4, ... $ とどんどん小さくして**有限**の値にするしかない．　<br>
ここでは$ f(x) = \sin{x} $ について $ h $ をどんどん小さくしていった結果微分がどのように変化するか考えていくことにする．

In [40]:
import math
# sin,cosが使えるようにmathをimportしておく

x0 = 1
d0 = math.cos(x0) # f'(x0)の理論解
f0 = round(math.sin(x0), 8) # round関数で8桁まで丸め込んでいる

for i in range(21):
    h = 2**(-i) # 刻み幅は 2^(-i) (累乗は"**"!!!)
    f1 = round(math.sin(x0 + h), 8) # f1同様8桁まで丸め込み
    d1 = (f1 - f0) / h #微分の定義
    print("h = 2^-" + str(i) + " -> " + str(math.fabs(d1-d0)) )
    # 理論値との差の絶対値、桁落ちしなければhが小さくなるほど小さくなるはずである
    i = i + 1

h = 2^-0 -> 0.4724758558681398
h = 2^-1 -> 0.22825428586813978
h = 2^-2 -> 0.1102477458681399
h = 2^-3 -> 0.053929425868139536
h = 2^-4 -> 0.026638945868139086
h = 2^-5 -> 0.01323478586814053
h = 2^-6 -> 0.006595425868135774
h = 2^-7 -> 0.003292385868137071
h = 2^-8 -> 0.0016424658681315485
h = 2^-9 -> 0.0008181458681106868
h = 2^-10 -> 0.00040854586809890847
h = 2^-11 -> 0.00020374586820670615
h = 2^-12 -> 8.086586804401108e-05
h = 2^-13 -> 3.990586829294429e-05
h = 2^-14 -> 4.201413302817869e-05
h = 2^-15 -> 4.201413302817869e-05
h = 2^-16 -> 0.0003696941346746918
h = 2^-17 -> 0.0010250541306917604
h = 2^-18 -> 0.0023357741227258977
h = 2^-19 -> 0.004957214135898003
h = 2^-20 -> 0.0049572141941056636


（round関数は一般的な四捨五入ではなく，近いほうの値に丸め込むという関数である．そのためC言語でfloat型の桁を指定してる参考サイトとは若干の値のずれはある）


$ h = 2^{-14} $ 以降の結果を見ると，$ h $ が小さくなっているのに $ d_1 $ と $ d_0 $ の差が大きくなっている様子が確認できる．これは $ f_0 $ や $ f_1 $ の値を小数第８位までに丸め込むことが原因で起きている **桁落ち**の代表例となっている．　<br>
講義の中でも解説があったようにfloat型の仮数部分は24bitsで　 $ f(x) = \sin(x) $ のような簡単な関数における微分は $ h = 2^{-20} $ 程度になっても桁落ちすることはないがより高次な項を含む関数や複雑な計算を要する場合，単に $ h $ を小さくすれば正確な微分値が求まるわけではないことが今回の課題を通して理解できた．

### 解析解に近づける

最後に8桁の丸め込みを実行しない正確な微分の演算をした場合のプログラム結果を示しておく．実は最初round関数の存在を知らずにプログラムを実行してしまい $ h $ が小さいほど正確に微分できてしまった… $ 2^{-20} = 0.00000095367... $ なのに結果を見たときはとっても驚いた．

In [15]:
import math
# mathをimportするのはおなじ
x0 = 1.0
d0 = math.cos(x0)
f0 = math.sin(x0)
# d0やｆ0をround関数で丸め込まずに計算する
for i in range(21):
    h = 2**(-i)
    f1 = math.sin(x0 + h)
    d1 = (f1 - f0) / h
    print("h = 2^-" + str(i) + " -> " + str(math.fabs(d1-d0)))
    i = i + 1
# 最初のプログラムと違うとのはd1とd0の桁数が8ではなく仮数部分の数字すべてが格納されているためより正確な数値になっていることである

h = 2^-0 -> 0.47247586385035456
h = 2^-1 -> 0.22825430227582388
h = 2^-2 -> 0.11024776767738098
h = 2^-3 -> 0.05392943153855034
h = 2^-4 -> 0.02663910012134485
h = 2^-5 -> 0.013234849721358355
h = 2^-6 -> 0.006595843010424485
h = 2^-7 -> 0.003292475538417161
h = 2^-8 -> 0.0016448699861526528
h = 2^-9 -> 0.0008220922622554916
h = 2^-10 -> 0.00041096035040899537
h = 2^-11 -> 0.0002054587177595657
h = 2^-12 -> 0.00010272399295407908
h = 2^-13 -> 5.136065483801744e-05
h = 2^-14 -> 2.5679992363247806e-05
h = 2^-15 -> 1.2839911691653327e-05
h = 2^-16 -> 6.419934110990511e-06
h = 2^-17 -> 3.2099562345955235e-06
h = 2^-18 -> 1.60496729639803e-06
h = 2^-19 -> 8.024873792145115e-07
h = 2^-20 -> 4.012037648770672e-07


実行結果をみれば明らかなように $ h = 2^{-20} $ でも誤差は約半分に減少していく一方で理論値にどんどん近づいていることが分かる．桁落ちしないから本来喜ぶべきなのだが課題内容に反した結果になり当時は非常に焦った．